In [21]:
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin, OneToOneFeatureMixin
from sklearn.pipeline import Pipeline
import seaborn as sns
from numpy.typing import NDArray
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [22]:
train_df = pd.read_csv('train.csv')
train_df.head()

,id,title,text,label,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,1,Regional African body says ready to work close...,JOHANNESBURG (Reuters) - The Southern African ...,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,New Report Says Trump Destroyed Scaramucci’s ...,Donald Trump s new pick for Communications Dir...,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Hamas chief in Gaza says Palestinian unity dea...,GAZA (Reuters) - Palestinian Islamist group Ha...,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Police fire tear gas at Congo opposition leade...,KINSHASA (Reuters) - Police fired tear gas to ...,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,PRESIDENT TRUMP Hits Back At Activist Judge On...,Judge Orrick in California ruled against Presi...,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
test_df = pd.read_csv('train.csv')
test_df.head()

,id,title,text,label,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,1,Regional African body says ready to work close...,JOHANNESBURG (Reuters) - The Southern African ...,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,New Report Says Trump Destroyed Scaramucci’s ...,Donald Trump s new pick for Communications Dir...,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Hamas chief in Gaza says Palestinian unity dea...,GAZA (Reuters) - Palestinian Islamist group Ha...,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Police fire tear gas at Congo opposition leade...,KINSHASA (Reuters) - Police fired tear gas to ...,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,PRESIDENT TRUMP Hits Back At Activist Judge On...,Judge Orrick in California ruled against Presi...,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
cat_df = train_df.select_dtypes(include=['object'])
num_df = train_df.select_dtypes(exclude=['object'])

In [25]:
train_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24360 entries, 0 to 24359
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           24360 non-null  int64  
 1   title        24360 non-null  object 
 2   text         24356 non-null  object 
 3   label        24348 non-null  object 
 4   Unnamed: 4   1 non-null      object 
 5   Unnamed: 5   1 non-null      object 
 6   Unnamed: 6   1 non-null      object 
 7   Unnamed: 7   1 non-null      object 
 8   Unnamed: 8   1 non-null      object 
 9   Unnamed: 9   1 non-null      object 
 10  Unnamed: 10  1 non-null      object 
 11  Unnamed: 11  1 non-null      object 
 12  Unnamed: 12  1 non-null      object 
 13  Unnamed: 13  1 non-null      object 
 14  Unnamed: 14  1 non-null      object 
 15  Unnamed: 15  1 non-null      object 
 16  Unnamed: 16  1 non-null      object 
 17  Unnamed: 17  1 non-null      object 
 18  Unnamed: 18  1 non-null      object 
 19  Unna

In [26]:
nan_df = (train_df.isnull().mean() * 100).reset_index()
nan_df.columns = ['column_name', 'percentage']
nan_df.sort_values('percentage', ascending=False, inplace=True)
nan_df.head(20)

,column_name,percentage
4,Unnamed: 4,99.995895
5,Unnamed: 5,99.995895
9,Unnamed: 9,99.995895
8,Unnamed: 8,99.995895
7,Unnamed: 7,99.995895
6,Unnamed: 6,99.995895
14,Unnamed: 14,99.995895
15,Unnamed: 15,99.995895
16,Unnamed: 16,99.995895
17,Unnamed: 17,99.995895


In [27]:
columns_to_drop = nan_df[nan_df['percentage'] > 90]['column_name']
train_df = train_df.drop(columns=columns_to_drop)
test_df = test_df.drop(columns=columns_to_drop)

In [28]:
cat_df = train_df.select_dtypes(include=['object'])
num_df = train_df.select_dtypes(exclude=['object'])

In [30]:
cat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24360 entries, 0 to 24359
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   24360 non-null  object
 1   text    24356 non-null  object
 2   label   24348 non-null  object
dtypes: object(3)
memory usage: 571.1+ KB


In [31]:
num_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24360 entries, 0 to 24359
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      24360 non-null  int64
dtypes: int64(1)
memory usage: 190.4 KB


In [32]:
print("Пропуски в train_df:", train_df.isnull().sum())
print("Пропуски в test_df:", test_df.isnull().sum())

Пропуски в train_df: id        0
title     0
text      4
label    12
dtype: int64
Пропуски в test_df: id        0
title     0
text      4
label    12
dtype: int64


In [33]:
train_df.head()

,id,title,text,label
0,1,Regional African body says ready to work close...,JOHANNESBURG (Reuters) - The Southern African ...,1
1,2,New Report Says Trump Destroyed Scaramucci’s ...,Donald Trump s new pick for Communications Dir...,0
2,3,Hamas chief in Gaza says Palestinian unity dea...,GAZA (Reuters) - Palestinian Islamist group Ha...,1
3,4,Police fire tear gas at Congo opposition leade...,KINSHASA (Reuters) - Police fired tear gas to ...,1
4,5,PRESIDENT TRUMP Hits Back At Activist Judge On...,Judge Orrick in California ruled against Presi...,0


In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from scipy.stats import randint, uniform
import numpy as np

def preprocess_text(text): 
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)  
    return text

for df in [train_df, test_df]:
    df['text'] = df['text'].fillna('').apply(preprocess_text)
    df['title'] = df['title'].fillna('').apply(preprocess_text)
    df['combined'] = df['title'] + ' ' + df['text']

train_df['label'] = train_df['label'].apply(
    lambda x: 1 if str(x).strip().lower() in ['1', 'true'] else 0
)

X_train, X_valid, y_train, y_valid = train_test_split(
    train_df['combined'],
    train_df['label'],
    test_size=0.2,
    random_state=42
)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=15000,
        ngram_range=(1, 2),
        stop_words='english'
    )),
    ('model', LGBMClassifier(
        random_state=42,
        n_jobs=-1,
        verbose=-1   
    ))
])

param_dist = {
    'tfidf__max_features': randint(10000, 20000),
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'model__n_estimators': randint(100, 300),
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__max_depth': randint(3, 7), 
}


search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=15, 
    scoring='f1_weighted',
    cv=3,
    n_jobs=-1,
    random_state=42,
    verbose=1
    )

search.fit(X_train, y_train)

print("\nЛучшие параметры:")
print(search.best_params_)

best_model = search.best_estimator_
valid_pred = best_model.predict(X_valid)
print(f"\nF1-score: {f1_score(y_valid, valid_pred, average='weighted'):.4f}")

test_pred = best_model.predict(test_df['combined'])
results = pd.DataFrame({'id': test_df['id'], 'label': test_pred})
results.to_csv('predictions_final.csv', index=False)

Fitting 3 folds for each of 15 candidates, totalling 45 fits

Лучшие параметры:
{'model__learning_rate': 0.1, 'model__max_depth': 6, 'model__n_estimators': 192, 'tfidf__max_features': 15390, 'tfidf__ngram_range': (1, 1)}


c:\artificial_intelligence\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



F1-score: 0.9856


c:\artificial_intelligence\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score 
from sklearn.pipeline import Pipeline

def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text) 
    return text

for df in [train_df, test_df]:
    df['text'] = df['text'].fillna('').apply(preprocess_text)
    df['title'] = df['title'].fillna('').apply(preprocess_text)
    df['combined'] = df['title'] + ' ' + df['text']

train_df['label'] = train_df['label'].apply(
    lambda x: 1 if str(x).strip().lower() in ['1', 'true'] else 0
)

X_train, X_valid, y_train, y_valid = train_test_split(
    train_df['combined'],
    train_df['label'],
    test_size=0.2,
    random_state=42
)

models = {
    'LightGBM': LGBMClassifier(
        random_state=42,
        n_jobs=-1,
        verbose=-1,
        n_estimators=192,   
        learning_rate=0.1,   
        max_depth=6
    ),
    'XGBoost': XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=7,
        random_state=42,
        n_jobs=-1,
        eval_metric='logloss',
        use_label_encoder=False
    ),
    'CatBoost': CatBoostClassifier(
        iterations=300,
        learning_rate=0.05,
        depth=7,
        random_state=42,
        verbose=0
    )
}

ensemble = VotingClassifier(
    estimators=[
        ('LightGBM', models['LightGBM']),
        ('XGBoost', models['XGBoost']),
        ('CatBoost', models['CatBoost'])
    ],
    voting='soft',  
    n_jobs=-1
)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=20000,
        ngram_range=(1, 2),
        stop_words='english'
    )),
    ('ensemble', ensemble)
])

print("Обучение ансамбля")
pipeline.fit(X_train, y_train)

valid_pred = pipeline.predict(X_valid)
print(f"\nF1-score: {f1_score(y_valid, valid_pred, average='weighted'):.4f}")

test_pred = pipeline.predict(test_df['combined'])
results = pd.DataFrame({'id': test_df['id'], 'label': test_pred})
#results.to_csv('predictions_v3.csv', index=False)

Обучение ансамбля


c:\artificial_intelligence\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



F1-score: 0.9858


c:\artificial_intelligence\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
